In [1]:
!pip3 install --user --force-reinstall 'google-cloud-aiplatform>=1.15' -q --no-warn-conflicts
!pip3 install --user kfp -q --no-warn-conflicts

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [2]:
import os

PROJECT_ID = ""

# Get your Google Cloud project ID from gcloud
if not os.getenv("IS_TESTING"):
    shell_output = !gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID: ", PROJECT_ID)

Project ID:  noted-casing-443608-r1


In [3]:
if PROJECT_ID == "" or PROJECT_ID is None:
    PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

In [4]:
#set your region 
REGION = "us-central1"  # @param {type: "string"}

In [5]:
#set timestamp to avoid collisions between multiple users

from datetime import datetime

TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")

In [6]:
#set cloud storage bucket 
BUCKET_NAME = "k21bucketdata"  # @param {type:"string"}
BUCKET_URI = f"gs://{BUCKET_NAME}"

In [7]:
# copy the data over to your cloud storage bucket
DATASET_URI = "gs://cloud-samples-data/vertex-ai/structured_data/player_data" 

!gsutil cp -r $DATASET_URI $BUCKET_URI

Copying gs://cloud-samples-data/vertex-ai/structured_data/player_data/data.csv [Content-Type=text/csv]...
Copying gs://cloud-samples-data/vertex-ai/structured_data/player_data/labels.csv [Content-Type=text/csv]...
/ [2 files][  5.1 MiB/  5.1 MiB]                                                
Operation completed over 2 objects/5.1 MiB.                                      


In [8]:
import logging
import os
import time

logger = logging.getLogger("logger")
logging.basicConfig(level=logging.INFO)

import kfp.compiler as compiler
# Pipeline Experiments
import kfp.dsl as dsl
# Vertex AI
from google.cloud import aiplatform as vertex_ai
from kfp.dsl import Artifact, Input, Metrics, Model, Output, component
from typing import NamedTuple

In [9]:
#import libraries and define constants
# Experiments

TASK = "regression"
MODEL_TYPE = "tensorflow"
EXPERIMENT_NAME = f"{PROJECT_ID}-{TASK}-{MODEL_TYPE}-{TIMESTAMP}"

# Pipeline
PIPELINE_URI = f"{BUCKET_URI}/pipelines"
TRAIN_URI = f"{BUCKET_URI}/player_data/data.csv"
LABEL_URI = f"{BUCKET_URI}/player_data/labels.csv"
MODEL_URI = f"{BUCKET_URI}/model"
DISPLAY_NAME = "experiments-demo-gaming-data"
BQ_DATASET = "player_data"
BQ_LOCATION = "US"  
VIEW_NAME = 'dataset_test'
PIPELINE_JSON_PKG_PATH = "experiments_demo_gaming_data.json"
PIPELINE_ROOT = f"gs://{BUCKET_URI}/pipeline_root"

In [10]:
#initialize vertex AI SDK 
vertex_ai.init(project=PROJECT_ID, staging_bucket=BUCKET_URI)

In [11]:
@component(
    packages_to_install=[
        "numpy==1.21.0",
        "pandas==1.3.5", 
        "scikit-learn==1.0.2",
        "tensorflow==2.9.0",
    ]
)
def custom_trainer(
    train_uri: str,
    label_uri: str,
    dropout_rate: float,
    learning_rate: float,
    epochs: int,
    model_uri: str,
    metrics: Output[Metrics], 
    model_metadata: Output[Model], 
    

):

    # import libraries
    import logging
    import uuid
    from pathlib import Path as path

    import pandas as pd
    import tensorflow as tf
    from tensorflow import keras
    from tensorflow.keras.models import Sequential
    from tensorflow.keras.layers import Dense
    from tensorflow.keras.layers import Dropout
    from tensorflow.keras.metrics import Metric 
    from sklearn.metrics import accuracy_score
    from sklearn.model_selection import train_test_split
    from sklearn.preprocessing import StandardScaler
    from sklearn.metrics import mean_absolute_error
    import numpy as np
    from math import sqrt
    import os
    import tempfile

    # set variables and use gcsfuse to update prefixes
    gs_prefix = "gs://"
    gcsfuse_prefix = "/gcs/"
    train_path = train_uri.replace(gs_prefix, gcsfuse_prefix)
    label_path = label_uri.replace(gs_prefix, gcsfuse_prefix)
    model_path = model_uri.replace(gs_prefix, gcsfuse_prefix)

    def get_logger():

        logger = logging.getLogger(__name__)
        logger.setLevel(logging.INFO)
        handler = logging.StreamHandler()
        handler.setFormatter(
            logging.Formatter("%(asctime)s - %(name)s - %(levelname)s - %(message)s")
        )
        logger.addHandler(handler)
        return logger

    def get_data(
        train_path: str, 
        label_path: str
    ) -> (pd.DataFrame): 
        
        
        #load data into pandas dataframe
        data_0 = pd.read_csv(train_path)
        labels_0 = pd.read_csv(label_path)
        
        #drop unnecessary leading columns
        
        data = data_0.drop('Unnamed: 0', axis=1)
        labels = labels_0.drop('Unnamed: 0', axis=1)
        
        #save as numpy array for reshaping of data 
        
        labels = labels.values
        data = data.values
    
        # Split the data
        labels = labels.reshape((labels.size,))
        train_data, test_data, train_labels, test_labels = train_test_split(data, labels, test_size=0.2, shuffle=True, random_state=7)
    
        #Convert data back to pandas dataframe for scaling
        
        train_data = pd.DataFrame(train_data)
        test_data = pd.DataFrame(test_data)
        train_labels = pd.DataFrame(train_labels)
        test_labels = pd.DataFrame(test_labels)
        
        #Scale and normalize the training dataset
        
        scaler = StandardScaler()
        scaler.fit(train_data)
        train_data = pd.DataFrame(scaler.transform(train_data), index=train_data.index, columns=train_data.columns)
        test_data = pd.DataFrame(scaler.transform(test_data), index=test_data.index, columns=test_data.columns)
        
        return train_data,train_labels, test_data, test_labels 
    
        """ Train your Keras model passing in the training data and values for learning rate, dropout rate,and the number of epochs """

    def train_model(
        learning_rate: float, 
        dropout_rate: float,
        epochs: float,
        train_data: pd.DataFrame,
        train_labels: pd.DataFrame):
 
        # Train tensorflow model
        param = {"learning_rate": learning_rate, "dropout_rate": dropout_rate, "epochs": epochs}
        model = Sequential()
        model.add(Dense(500, input_dim=train_data.shape[1], activation= "relu"))
        model.add(Dropout(param['dropout_rate']))
        model.add(Dense(100, activation= "relu"))
        model.add(Dense(50, activation= "relu"))
        model.add(Dense(1))
            
        model.compile(
        tf.keras.optimizers.Adam(learning_rate= param['learning_rate']),
        loss='mse',
        metrics=[tf.keras.metrics.RootMeanSquaredError(),tf.keras.metrics.MeanAbsoluteError()])
        
        model.fit(train_data, train_labels, epochs= param['epochs'])
        
        return model

    # Get Predictions
    def get_predictions(model, test_data):

        dtest = pd.DataFrame(test_data)
        pred = model.predict(dtest)
        return pred

    # Evaluate predictions with MAE
    def evaluate_model_mae(pred, test_labels):
        
        mae = mean_absolute_error(test_labels, pred)
        return mae
    
    # Evaluate predictions with RMSE
    def evaluate_model_rmse(pred, test_labels):

        rmse = np.sqrt(np.mean((test_labels - pred)**2))
        return rmse    
 
    
    #Save your trained model in GCS     
    def save_model(model, model_path):

        model_id = str(uuid.uuid1())
        model_path = f"{model_path}/{model_id}"        
        path(model_path).parent.mkdir(parents=True, exist_ok=True)
        model.save(model_path + '/model_tensorflow')

        
    # Main ----------------------------------------------
    
    train_data, train_labels, test_data, test_labels = get_data(train_path, label_path)
    model = train_model(learning_rate, dropout_rate, epochs, train_data,train_labels )
    pred = get_predictions(model, test_data)
    mae = evaluate_model_mae(pred, test_labels)
    rmse = evaluate_model_rmse(pred, test_labels)
    save_model(model, model_path)

    # Metadata ------------------------------------------

    #convert numpy array to pandas series
    mae = pd.Series(mae)
    rmse = pd.Series(rmse)

    #log metrics and model artifacts with ML Metadata. Save metrics as a list. 
    metrics.log_metric("mae", mae.to_list()) 
    metrics.log_metric("rmse", rmse.to_list()) 
    model_metadata.uri = model_uri

/opt/conda/lib/python3.10/site-packages/kfp/dsl/component_decorator.py:119: FutureWarning: Python 3.7 has reached end-of-life. The default base_image used by the @dsl.component decorator will switch from 'python:3.7' to 'python:3.8' on April 23, 2024. To ensure your existing components work with versions of the KFP SDK released after that date, you should provide an explicit base_image argument and ensure your component works as intended on Python 3.8.
  return component_factory.create_component_from_func(


In [12]:
@dsl.pipeline(name="gaming-custom-training-pipeline")
def pipeline(
    train_uri: str,
    label_uri: str,
    dropout_rate: float,
    learning_rate: float,
    epochs: int,
    model_uri: str,
):
    # Pass arguments as keyword arguments
    custom_trainer(
        train_uri=train_uri,
        label_uri=label_uri,
        dropout_rate=dropout_rate,
        learning_rate=learning_rate,
        epochs=epochs,
        model_uri=model_uri
    )


In [13]:
#compile our pipeline
compiler.Compiler().compile(pipeline_func=pipeline, package_path="gaming_pipeline.json")

In [14]:
runs = [
    {"dropout_rate": 0.001, "learning_rate": 0.001,"epochs": 20},
    {"dropout_rate": 0.002, "learning_rate": 0.002,"epochs": 25},
    {"dropout_rate": 0.003, "learning_rate": 0.003,"epochs": 30},
    {"dropout_rate": 0.004, "learning_rate": 0.004,"epochs": 35},
    {"dropout_rate": 0.005, "learning_rate": 0.005,"epochs": 40},
]

In [15]:
for i, run in enumerate(runs):

    job = vertex_ai.PipelineJob(
        display_name=f"{EXPERIMENT_NAME}-pipeline-run-{i}",
        template_path="gaming_pipeline.json",
        pipeline_root=PIPELINE_URI,
        parameter_values={
            "train_uri": TRAIN_URI,
            "label_uri": LABEL_URI,
            "model_uri": MODEL_URI,
            **run,
        },
    )
    job.submit(experiment=EXPERIMENT_NAME)

INFO:google.cloud.aiplatform.pipeline_jobs:Creating PipelineJob
INFO:google.cloud.aiplatform.pipeline_jobs:PipelineJob created. Resource name: projects/83022982256/locations/us-central1/pipelineJobs/gaming-custom-training-pipeline-20241212052025
INFO:google.cloud.aiplatform.pipeline_jobs:To use this PipelineJob in another session:
INFO:google.cloud.aiplatform.pipeline_jobs:pipeline_job = aiplatform.PipelineJob.get('projects/83022982256/locations/us-central1/pipelineJobs/gaming-custom-training-pipeline-20241212052025')
INFO:google.cloud.aiplatform.pipeline_jobs:View Pipeline Job:
https://console.cloud.google.com/vertex-ai/locations/us-central1/pipelines/runs/gaming-custom-training-pipeline-20241212052025?project=83022982256
INFO:google.cloud.aiplatform.metadata.experiment_resources:Associating projects/83022982256/locations/us-central1/pipelineJobs/gaming-custom-training-pipeline-20241212052025 to Experiment: noted-casing-443608-r1-regression-tensorflow-20241212052000
INFO:google.cloud.

In [16]:
# see state/status of all the pipeline runs

vertex_ai.get_experiment_df(EXPERIMENT_NAME)

,experiment_name,run_name,run_type,state,param.train_uri,param.learning_rate,param.epochs,param.label_uri,param.model_uri,param.dropout_rate
0,noted-casing-443608-r1-regression-tensorflow-2...,gaming-custom-training-pipeline-20241212052033,system.PipelineRun,RUNNING,gs://k21bucketdata/player_data/data.csv,0.005,40.0,gs://k21bucketdata/player_data/labels.csv,gs://k21bucketdata/model,0.005
1,noted-casing-443608-r1-regression-tensorflow-2...,gaming-custom-training-pipeline-20241212052031,system.PipelineRun,RUNNING,gs://k21bucketdata/player_data/data.csv,0.004,35.0,gs://k21bucketdata/player_data/labels.csv,gs://k21bucketdata/model,0.004
2,noted-casing-443608-r1-regression-tensorflow-2...,gaming-custom-training-pipeline-20241212052029,system.PipelineRun,RUNNING,gs://k21bucketdata/player_data/data.csv,0.003,30.0,gs://k21bucketdata/player_data/labels.csv,gs://k21bucketdata/model,0.003
3,noted-casing-443608-r1-regression-tensorflow-2...,gaming-custom-training-pipeline-20241212052028,system.PipelineRun,RUNNING,gs://k21bucketdata/player_data/data.csv,0.002,25.0,gs://k21bucketdata/player_data/labels.csv,gs://k21bucketdata/model,0.002
4,noted-casing-443608-r1-regression-tensorflow-2...,gaming-custom-training-pipeline-20241212052025,system.PipelineRun,RUNNING,gs://k21bucketdata/player_data/data.csv,0.001,20.0,gs://k21bucketdata/player_data/labels.csv,gs://k21bucketdata/model,0.001


In [17]:
#check on current status
while True:
    pipeline_experiments_df = vertex_ai.get_experiment_df(EXPERIMENT_NAME)
    if all(
        pipeline_state != "COMPLETE" for pipeline_state in pipeline_experiments_df.state
    ):
        print("Pipeline runs are still running...")
        if any(
            pipeline_state == "FAILED"
            for pipeline_state in pipeline_experiments_df.state
        ):
            print("At least one Pipeline run failed")
            break
    else:
        print("Pipeline experiment runs have completed")
        break
    time.sleep(60)

Pipeline runs are still running...
Pipeline runs are still running...
Pipeline runs are still running...
Pipeline runs are still running...
Pipeline experiment runs have completed


In [18]:
# Call the pipeline runs based on the experiment run name
pipeline_experiments_df = vertex_ai.get_experiment_df(EXPERIMENT_NAME)
job = vertex_ai.PipelineJob.get(pipeline_experiments_df.run_name[0])
print(job.resource_name)
print(job._dashboard_uri())

projects/83022982256/locations/us-central1/pipelineJobs/gaming-custom-training-pipeline-20241212052033
https://console.cloud.google.com/vertex-ai/locations/us-central1/pipelines/runs/gaming-custom-training-pipeline-20241212052033?project=83022982256


In [19]:
# wait 60 seconds and view state again
import time
time.sleep(60)
vertex_ai.get_experiment_df(EXPERIMENT_NAME)

,experiment_name,run_name,run_type,state,param.epochs,param.model_uri,param.learning_rate,param.train_uri,param.label_uri,param.dropout_rate,metric.mae,metric.rmse
0,noted-casing-443608-r1-regression-tensorflow-2...,gaming-custom-training-pipeline-20241212052033,system.PipelineRun,COMPLETE,40.0,gs://k21bucketdata/model,0.005,gs://k21bucketdata/player_data/data.csv,gs://k21bucketdata/player_data/labels.csv,0.005,[1.15807511715334],[1.4763584595078787]
1,noted-casing-443608-r1-regression-tensorflow-2...,gaming-custom-training-pipeline-20241212052031,system.PipelineRun,COMPLETE,35.0,gs://k21bucketdata/model,0.004,gs://k21bucketdata/player_data/data.csv,gs://k21bucketdata/player_data/labels.csv,0.004,[1.462405707730188],[1.7820092257329112]
2,noted-casing-443608-r1-regression-tensorflow-2...,gaming-custom-training-pipeline-20241212052029,system.PipelineRun,COMPLETE,30.0,gs://k21bucketdata/model,0.003,gs://k21bucketdata/player_data/data.csv,gs://k21bucketdata/player_data/labels.csv,0.003,[1.175003677841041],[1.5155264234475607]
3,noted-casing-443608-r1-regression-tensorflow-2...,gaming-custom-training-pipeline-20241212052028,system.PipelineRun,COMPLETE,25.0,gs://k21bucketdata/model,0.002,gs://k21bucketdata/player_data/data.csv,gs://k21bucketdata/player_data/labels.csv,0.002,[0.7893661149870809],[1.0727045200107925]
4,noted-casing-443608-r1-regression-tensorflow-2...,gaming-custom-training-pipeline-20241212052025,system.PipelineRun,COMPLETE,20.0,gs://k21bucketdata/model,0.001,gs://k21bucketdata/player_data/data.csv,gs://k21bucketdata/player_data/labels.csv,0.001,[0.8229515408336361],[1.1153459315521985]
